## Python Environment
Run the cells below to make sure that you are in correct python environment to run the project. 

*Required*
- **Python 3.9.***
- **Conda ENV - Urban**

In [1]:
!python -V
import os
print("Conda ENV - ", os.environ['CONDA_DEFAULT_ENV'])

Python 3.9.13


KeyError: 'CONDA_DEFAULT_ENV'

## Python Libraries
All the libraries imported are installed within the conda environment. If the library is not installed, it will show an error. 

In [2]:
import Landuse
#from Demand import Demand
import Network
import Assignment
from Visualization import Visualize 
from Effect import Effect
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'geopandas'

## Variable setting 

In [ ]:
######### VARIABLES ############
fuel_cost = 12      # fuel cost 12 sek/liter
fuel_con = 0.085    # fuel consumption is 0.085 l/km
cost_km = fuel_cost * fuel_con 
carparking = 5      # Cost of Parking
transitprice = 15   # Cost of Transit

## Parameter setting

In [ ]:
######### PARAMETERS ############
alpha = 0.45        # constant for car mode
beta_car = -0.08  # parameter for car travel time
gamma_car = -0.05   # parameter for car travel cost
beta_inv = -0.05    # parameter for transit in-vehile travel time
beta_wait = -0.08    # parameter for transit total waiting time
gamma_pt = -0.05  # parameter for transit ticket price
alpha_slow = 0.1  #parameter for slow mode
phi_dist = -0.5    # parameter for slow mode time
mu = 0.5            # Logsum weight
theta = 1           # parameter for destination choice 
param = [alpha, beta_car, gamma_car, beta_inv, beta_wait, gamma_pt, alpha_slow, phi_dist, mu, theta] # to pass the parameter value to demand function as a list
vot_car = beta_car/gamma_car        # value of time for car
vot_in = beta_inv/gamma_pt         # value of in-vehile travel time for transit
vot_wait = beta_wait/gamma_pt      # value of total waiting time for transit
constant = 0.2      # alternative specific constant for owning a car 
income = 0.003      # parameter for income 
dummy = -0.5        # dummy for residency in the inner city
param_carown = [constant, income, dummy] # to pass the parameter value to landuse function as a list

## Import data 

In [ ]:
# Load landuse and zone information
landuse, zones = Landuse.get_landuse_and_zones(param_carown)

# Origin and Destination dictionary
origin      = dict(zip(landuse.area, landuse.index))
destination = origin.copy()

# Create Graph network for car and transit
G_car = Network.RoadNetwork(zones.transpose())
G_pt = Network.TransitNetwork(zones.transpose())

## Initialize the network

In [ ]:
# Initialize the network
demand = np.ones((len(landuse), len(landuse))) 
demand = np.asmatrix(demand)
G_start_car = Assignment.RouteAssignment(np.asarray(demand), G_car, origin, destination,vot_car,cost_km) 
G_start_pt = Assignment.TransitAssignment(np.asarray(demand), G_pt, origin, destination,vot_in,vot_wait)
diagnal_dist = np.sqrt(landuse['area_2'])/1000 # to approximate the distance of inne-zone trips
car_time, car_cost, dist = Assignment.Skim_car(G_start_car, origin, destination, diagnal_dist)
invt, waitt = Assignment.Skim_pt(G_start_pt, origin, destination)

## Demand funtion example with fixed probability

In [ ]:
def Demand_fix (car_time, car_cost, car_park, inv_time, wait_time, pt_price, dist, parameter, pop, emp, own):
    
    # the following parameters will be used for utility functions
    alpha = parameter[0]            
    beta_car = parameter[1]
    gamma_car = parameter[2]
    beta_inv = parameter[3]
    beta_wait = parameter[4]
    gamma_pt = parameter[5] 
    alpha_dist = parameter[6]
    phi_dist = parameter[7]
    mu = parameter[8]
    theta = parameter[9] 

    
    pop = np.asmatrix(pop).transpose()
    own = np.asmatrix(own).transpose()
    emp = np.asmatrix(emp)
    
    # assuming a fixed probability 
    Pcar = np.ones((len(pop), len(pop))) *(0.4/len(pop))
    Ppt  = np.ones((len(pop), len(pop)))  *(0.4/len(pop))
    Pslow= np.ones((len(pop), len(pop)))*(0.2/len(pop))
    
    # calculating the demand
    Vol_car  = np.multiply(pop,Pcar)
    Vol_pt   = np.multiply(pop,Ppt)
    Vol_slow = np.multiply(pop,Pslow)
    EU = 0

    return (Vol_car, Vol_pt, Vol_slow, EU)

## Equilibrium 

In [ ]:
## Suggested Equilibrium cell (also based on the Project file)
# calculate demand
maxiter = 100       # maxium number of iterations in finding the equilibrium
maxdiff = 0.01      # or if the travel time converges to the point where difference in travel time between iterations is smaller than 0.01
for iter in range(maxiter):
    print("\n")
    car_time_old = car_time
    car_cost_old = car_cost
    
    # Here, you can change to your demand funtion
    v_car, v_pt, v_slow, EU = Demand_fix(car_time,
                         car_cost,
                         carparking,
                         invt,
                         waitt,
                         transitprice,
                         dist,
                         param,
                         np.array(landuse['pop']),
                         np.array(landuse['emp']),
                         np.array(landuse['car_ownership']))  
    # assign the traffic based on the demand to get the new car time, cost, and distance
    G_next_car = Assignment.RouteAssignment(np.asarray(v_car), G_car, origin, destination,vot_car,cost_km) 
    G_next_pt = Assignment.TransitAssignment(np.asarray(v_pt), G_pt, origin, destination,vot_in,vot_wait)
    car_time, car_cost, car_dist = Assignment.Skim_car(G_next_car, origin, destination, diagnal_dist)

    # calculate car_time values with 0.2 share of new values (from assignemet.skim_car) and 
    # 0.8 share of the old car time values
    car_time = car_time * 0.2 + car_time_old * 0.8
    car_cost = car_cost * 0.2 + car_cost_old * 0.8
   
    # define the average error (e.g. 0.2) for convergence, and if it is met, make (stop = 1)
    diff = (np.mean((np.abs(car_time-car_time_old))))
    print(f"Iteration: {iter}\tDifference in travel time:{diff}")
    if (diff < maxdiff) or (iter>=maxiter):
        break

## RESULT
The cells shows the result of the 4-step model. 

In [ ]:
# output result in a table, you can specify what output might be useful 
# Hint: what is the total number of trips?
Effect(v_car, v_pt, v_slow, dist, carparking, transitprice, EU)

In [ ]:
# below the car flow and transit flow are plotted in seperate layer, you can activate/deactivate layers 
Visualize(G_next_car, G_next_pt, zones, title="REULST FOR 4-STEP MODEL")

## Task I
The main task is to construct a nested choice model, assuming that individuals make a joint decision on the destination choice and mode choice. <br>

Assume that there are three alternative modes of transport  that can be used to travel between two zones, $i$ and $j$, that the the deterministic utility for respectively mode are given by:
\begin{align}
V^i_{j, \textrm{car}} &= \alpha_\text{car} + \beta_{\textrm{car}} t^i_{j,\text{car}} + \gamma_{\textrm{car}} c^i_{j,\text{car}} \\%+ \theta \cdot \ln(N^j_\text{employed}) \\ 
V^i_{j,\textrm{pt}} &= \beta_{\textrm{inv}} t^i_{j,\textrm{inv}} + \beta_{\textrm{wait}} t^i_{j,\textrm{wait}} + \gamma_{\textrm{pt}} c^i_{j,\text{pt}} \\%+ \theta \cdot  \ln(N^j_\text{employed}), \\ 
V^i_{j, \textrm{slow}} &= \alpha_\text{slow} + \phi d^i_{j} \\%+  \theta \cdot  \ln(N^j_\text{employed})
\end{align}

<br>
$t^i_{j,\textrm{car}} - \text{car travel time: } \text{tt_car}\\
 c^i_{j,\text{car}} - \text{car travel cost: } \text{c_car: fuel cost and parking cost}\\
 t^i_{j,\textrm{inv}} - \text{pt in-vehicle travel time: } \text{tt_inv_pt}\\
 t^i_{j,\textrm{wait}} - \text{pt waiting time: } \text{tt_wait_tt}\\
 c^i_{j,\text{pt}} - \text{pt cost: } \text{c_pt}\\
 d^i_{j} - \text{distace} \\$
<br>

The deterministic utility of destination choice is formulated as
\begin{align}
V_{j}=\theta ln (N_{employed}^j)
\end{align} 

As the destination choice is assumed made at the upper level, meaning that the utility depends also on how "easy" to travel to the destination. The properties of the mode chice are then propagated to the destnation choice through the expected maximum utility (log-sum) of all modes:    
\begin{align} 
I^i_j = ln \sum_m e^{V^i_{j,m}/ \mu}
\end{align} 
multiplyed by a scale parameter $\mu$, $\mu I^i_j$ is the expected maximum utility of the lower nest of mode choices. 

The probability of choosing a destination $j$ is given by
\begin{align}
P(j|i)= \frac{e^{V^i_j+\mu L^i_j}}{\sum_{j'\in J} e^{V^i_j+\mu I^i_{j'}}}           
\end{align} 

Note that the mode choice probability need to be rewriten with the scale parameter
\begin{align}
P(m|j,i)=\frac{e^{V^i_{j,m}/ \mu}}{\sum_{m'\in M} e^{V^i_{j,m'}/ \mu}}
\end{align} 

The probability of choosing mode $m$ from origin $i$ to destination $j$ is a joint probability, the product of the marginal probability $P(j)$ and the conditional probability $P(m|j) $ 
\begin{align}
P(j,m|i)=P(j|i)* P(m|j,i)            
\end{align} 



In [ ]:
def Demand_nestlogit (car_time, car_cost, car_park, inv_time, wait_time, pt_price, dist, parameter, pop, emp, own):
    
    # the following parameters will be used for utility functions
    alpha = parameter[0]            
    beta_car = parameter[1]
    gamma_car = parameter[2]
    beta_inv = parameter[3]
    beta_wait = parameter[4]
    gamma_pt = parameter[5]
    alpha_dist = parameter[6]
    phi_dist = parameter[7]
    mu = parameter[8]
    theta = parameter[9] 
    
        
    pop = np.asmatrix(pop).transpose()
    own = np.asmatrix(own).transpose()
    emp = np.asmatrix(emp)
    
   
    # To input utility functions for each mode 
    Vcar  = #utitlity for car
    Vpt   = #utitlity for pt
    Vslow = #utitlity for slow
    # To input the utility function for attraction 
    Vj  = 
   
    
    # To calculate the condition probability for each mode
    # Note that the parameter mu is being used in the nested logit probabilty
    denom  = 
    Pcar  =
    Ppt   =
    Pslow = 
    
    # To calculate the marginal probability for going to j from i 
    Lj    =    #Expected maximum utilities from mode choice
    Pj    =    #marginal probability 
       
    # Hint: use print() to check if the marginal probability sum for each origin 
    
    # To calculate the joint probability 
    Pcar_j  =
    Ppt_j  =
    Pslow_j  =
    
    # Hint: use print() to check if the koint probability sum for all modes and each origin
    
    
    # calculating the demand
    Vol_car  = np.multiply(pop,Pcar)
    Vol_pt   = np.multiply(pop,Ppt)
    Vol_slow = np.multiply(pop,Pslow)    
    EU = 0

    return (Vol_car, Vol_pt, Vol_slow, EU)

## Task II
Now we want to divide the population into two categories: individuals who have access to a car and those who do not. For individuals with access to a car, the set of possible 
transport modes is $ \lbrace car,pt,slow \rbrace $. Without access to a car, the set is $\lbrace pt,slow \rbrace $. 
The demand is the total demand of these two groups of population. <br>

The expected utility $EU$ can be used as a welfare measure when the change between two different scenarios is analysed. If cost enters the utility functions $V$, the difference in expected utility  $EU$ between two different scenarios can be translated into a monetary terms and used as a measure for consumer surplus. 
Please write the expected maximum utility from the joint decision model. 

In [ ]:
def Demand (car_time, car_cost, car_park, inv_time, wait_time, pt_price, dist, parameter, pop, emp, own):
    
    # the following parameters will be used for utility functions
    alpha = parameter[0]            
    beta_car = parameter[1]
    gamma_car = parameter[2]
    beta_inv = parameter[3]
    beta_wait = parameter[4]
    gamma_pt = parameter[5]
    alpha_dist = parameter[6]
    phi_dist = parameter[7]
    mu = parameter[8]
    theta = parameter[9]   
    
        
    pop = np.asmatrix(pop).transpose()
    own = np.asmatrix(own).transpose()
    emp = np.asmatrix(emp)
    
   
    # To input utility functions for each mode 
    # To input utility functions for each mode 
    Vcar  = #utitlity for car
    Vpt   = #utitlity for pt
    Vslow = #utitlity for slow
    # To input the utility function for attraction 
    Vj  = 
   
    #With car
    # To calculate the condition probability for each mode
    # Note that the parameter mu is being used in the nested logit probabilty
    denom  = 
    Pcar  =
    Ppt   =
    Pslow = 
    
    # To calculate the marginal probability for going to j from i 
    Lj    =   #Expected maximum utilities from mode choice
    Pj    =    #marginal probability 
    
    # Hint: use print() to check if the marginal probability sum for each origin 
    
    # To calculate the joint probability 
    Pcar_j  =
    Ppt_j   =
    Pslow_j =
    
    # Hint: use print() to check if the joint probability sum for all modes and each origin
    
    
    # Without car
    # To calculate the condition probability for each mode
    # Note that the parameter mu is being used in the nested logit probabilty
    denom_no = 
    Ppt_no   =
    Pslow_no =
    
    # To calculate the marginal probability for going to j from i 
    Lj_no     =    #Expected maximum utilities from mode choice
    Pj_no    =    #marginal probability 
    
    # Hint: use print() to check if the marginal probability sum for each origin 
   
    
    # To calculate the joint probability
    Ppt_j_no   =
    Pslow_j_no =
    
    # Hint: use print() to check if the koint probability sum for all modes and each origin
    
    
    
    # calculating the travel demand, using the values in matrix 'own'
    Vol_car  = 
    Vol_pt   = 
    Vol_slow = 
    
    # calculating the expected utility for the two groups
    EU_yes = 
    EU_no  =
    EU     =


    return (Vol_car, Vol_pt, Vol_slow, EU)